# CTIPe


The kamodofied CTIPe requires `netcdf4` (and numpy, scipy, and pandas, but those are required by kamodo)

    pip install netcdf4

!!! warning
    This is a work in progress!

In [1]:
%load_ext autoreload
%autoreload 2

In [302]:
from netCDF4 import Dataset
import numpy as np
import os
import scipy
from scipy import interpolate
from scipy.interpolate import RegularGridInterpolator
import math
import time
import kamodo
from datetime import datetime,timezone
import pandas as pd
from plotly.offline import init_notebook_mode, iplot, plot
init_notebook_mode(connected = True)

In [315]:
#filename = '/Users/lrastaet/Kamodo_data/CTIPe/2020-05-04-plot-density.nc' # each CTIPe run has plot-density, plot-height, plot-neutral and plot-plasma files for each date
#filename = '/Users/lrastaet/Kamodo_data/CTIPe//Sujay_Pal_042620_IT_1/2015-03-20-plot-neutral.nc'
filename = '/Users/lrastaet/Kamodo_data/CTIPe//Sujay_Pal_042620_IT_1/2015-03-20-plot-plasma.nc'
#filename = '/Users/lrastaet/Kamodo_data/CTIPe//Sujay_Pal_042620_IT_1/2015-03-20-plot-height.nc'
#filename= '/Users/lrastaet/Kamodo_data/CTIPe/Sujay_Pal_042620_IT_1/2015-03-20-plot-density.nc' 

In [316]:
dt=datetime.strptime("2015/06/12 05:07:01",'%Y/%m/%d %H:%M:%S') #,tzinfo=timezone.utc)

In [317]:
delta_t=dt-datetime(1970,1,1)
print(delta_t.total_seconds())

1434085621.0


In [318]:
from readers.ctipe_singletime import CTIPe,ctipe_kamodo_variable_names
from readers.geopack_transform import seconds_from_19700101
def seconds_from_19700101_to_DateTime(seconds):
    from datetime import datetime,timezone,timedelta
    time0=datetime(1970,1,1,0,0,0,tzinfo=timezone.utc)+timedelta(seconds=seconds)
    return time0
from tiegcm.util import datetime_to_epoch

By default, the kamodofied CTIPe reader will load all 3d and 4d variables from the model. For performance reasons, you may want to only include certain variables for analysis. To do so, pass a list of variables in addition to the filename:

In [314]:
ctipe = CTIPe(filename,variables_requested=['T'],date="2015/03/20",time="01:15:00",debug=False)
#ctipe = CTIPe(filename,debug=False)
#ctipe = CTIPe_Kamodo(filename, ['TN', 'UN', 'VN', 'EFLUX', 'Z'])

/Users/lrastaet/Kamodo_data/CTIPe/Sujay_Pal_042620_IT_1/2015-03-20-plot-density.nc
/Users/lrastaet/Kamodo_data/CTIPe/Sujay_Pal_042620_IT_1/2015-03-20-plot-height.nc
/Users/lrastaet/Kamodo_data/CTIPe/Sujay_Pal_042620_IT_1/2015-03-20-plot-neutral.nc
/Users/lrastaet/Kamodo_data/CTIPe/Sujay_Pal_042620_IT_1/2015-03-20-plot-plasma.nc
[1.4268105e+09 1.4268114e+09 1.4268123e+09 1.4268132e+09 1.4268141e+09
 1.4268150e+09 1.4268159e+09 1.4268168e+09 1.4268177e+09 1.4268186e+09
 1.4268195e+09 1.4268204e+09 1.4268213e+09 1.4268222e+09 1.4268231e+09
 1.4268240e+09 1.4268249e+09 1.4268258e+09 1.4268267e+09 1.4268276e+09
 1.4268285e+09 1.4268294e+09 1.4268303e+09 1.4268312e+09 1.4268321e+09
 1.4268330e+09 1.4268339e+09 1.4268348e+09 1.4268357e+09 1.4268366e+09
 1.4268375e+09 1.4268384e+09 1.4268393e+09 1.4268402e+09 1.4268411e+09
 1.4268420e+09 1.4268429e+09 1.4268438e+09 1.4268447e+09 1.4268456e+09
 1.4268465e+09 1.4268474e+09 1.4268483e+09 1.4268492e+09 1.4268501e+09
 1.4268510e+09 1.4268519e+09 1.

In [272]:
ctipe

CTIPe([(rho(xvec),
        <function readers.ctipe_singletime.CTIPe.register_4d_density_variable.<locals>.interpolator(xvec)>),
       (rho,
        <function readers.ctipe_singletime.CTIPe.register_4d_density_variable.<locals>.interpolator(xvec)>),
       (H(xvec),
        <function readers.ctipe_singletime.CTIPe.register_4d_neutral_variable.<locals>.interpolator(xvec)>),
       (H,
        <function readers.ctipe_singletime.CTIPe.register_4d_neutral_variable.<locals>.interpolator(xvec)>)])

In [273]:
ctipe.set_plot('LonLat',cutV=15,latrange=dict(min=-20,max=70,n=31),lonrange=dict(min=0,max=120,n=61))
plot=ctipe.get_plot('H',colorscale="Rainbow")

iplot(plot)

Time resetting plot and precomputing interpolations: 0.0003 seconds


In [274]:
ctipe.set_plot('LatIP',cutV=50,latrange=dict(min=80,max=90),hrange=dict(min=5))
plot=ctipe.get_plot('rho',colorscale="Rainbow")
iplot(plot)

Time resetting plot and precomputing interpolations: 0.0003 seconds
xrange:  {'min': 80.0, 'max': 90.0, 'n': 31}
yrange:  {'min': 5.0, 'max': 15.0, 'n': 15}
aspect ratio: x= 1  y= 1.0


ctipe.set_plot('LonIP')
plot=ctipe.get_plot('rho',colorscale="Rainbow")
iplot(plot)

In [126]:
ctipe.set_plot('LonIP',cutV=0)
plot=ctipe.get_plot('rho',colorscale="Rainbow")
iplot(plot)

Time resetting plot and precomputing interpolations: 0.0003 seconds


In [ ]:
print(ctipe['H'].data[0,:,0,0])

In [ ]:
H=ctipe['H'].data
#hintp=RegularGridInterpolator((ctipe._time,H[0,:,0,0].flatten(),ctipe._lat_density,ctipe._lon_density),H)
hintp=HeightInterpolator((ctipe._time,H[0,:,0,0].flatten(),ctipe._lat_density,ctipe._lon_density),H)


In [ ]:
print(hintp([ctipe._time[0],125000.,10.3,50.3],method='linear'))


In [ ]:
## comparison test with IDL in run 
## IDL> print,get_pointdata([50.,0.,10.])
## % Compiled module: GET_POINTDATA_3D.
##  5.07806e-10      188.940      19.6322     -6.30557      28.9232      1.83336      595.492  7.52705e+10  1.99925e-05   0.00973094
##    -0.985651  1.08981e+16  1.08948e+14  4.56949e+15  1.68988e+12  1.35428e+10      6169.57  7.52248e+09      29051.7  5.42052e+10
##  1.73583e+06  3.07283e-05  1.89448e-06     -3.32263      2.88644  1.39743e-06      0.00000      0.00000 -0.000311317 -0.000303866
## -0.000346761 -0.000338463      36.2960      0.00000  1.38551e+12      260.000
##IDL> print,qusntities
##% PRINT: Variable is undefined: QUSNTITIES.
##% Execution halted at: $MAIN$              1 /ccmcshare/IDL/3DView/ak3me.pro
##IDL> print,quantities
##rho H Rmt Vn_lat Vn_lon Vn_IP T_n N_e Psolar Pjoule Prad N_O N_O2 N_N2 N_NO N_NO+ N_N2+ N_O2+ N_N+ N_O+ N_H+ sigma_P sigma_H Vi_lat
##Vi_lon Wjoule Win En_avg E140_theta E300_theta E140_lambda E300_lambda TEC_above_140km Vi_IP NmF2 HmF2
##IDL> print,year,month,time,put_time(time)
##        2015           3   9.9999999e-0900:00:00.0000

In [ ]:
testdata=dict()
testdata['ip']=5.5
testdata['Height']='125'
testdata['date']='2015/3/20'
testdata['time']='00:15:00'
testdata['lon']=50.
testdata['lat']=10.
testdata['rho']='1.49067e-07  1.64051e-08'
testdata['H']='105.230      125.000'
testdata['Rmt']='26.3395      23.9059'
testdata['Vn_lat']='-0.0601084      43.4968'
testdata['Vn_lon']='-23.2656     -3.15513'
testdata['Vn_IP']='-0.0434272   -0.0818311'
testdata['T']='298.432      430.817'  #T_n
testdata['N_e']='1.06982e+08  5.90486e+08'
testdata['Psolar']='1.10576e-06  2.72283e-06'
testdata['Pjoule']='4.78753e-05  0.000398958'
testdata['Prad']=' -0.210089     -1.06214'
testdata['N_O']='4.70951e+17  1.42260e+17'
testdata['N_O2']='3.72280e+17  1.27961e+16'
testdata['N_N2']='2.51143e+18  2.56911e+17'
testdata['N_NO']='6.02975e+13  3.22197e+13'
testdata['N_NOplus']='1.06908e+08  5.04418e+08'
testdata['N_N2plus']='4.83005     0.987384'
testdata['N_O2plus']='74194.3  8.11062e+07'
testdata['N_Nplus']='0.615991     0.795714'
testdata['N_Oplus']='3.41415  4.96166e+06' #N_O+
testdata['N_Hplus']='0.00000      32.4158'
testdata['sigma_P']='9.66247e-07  5.87935e-06'
testdata['sigma_H']='9.17236e-06  8.91819e-06'
testdata['Vi_lat']='-0.723088      40.6683'
testdata['Vi_lon']='-23.9126     -9.70373'
testdata['Wjoule']='5.02903e-07  5.02903e-07'
testdata['Win ']='0.00000      0.00000'
testdata['En_avg']='0.00000      0.00000'
testdata['E_theta140km']='-0.000191656 -0.000191656'
testdata['E_theta300km']='-0.000187069 -0.000187069'
testdata['E_lambda140km']='-0.000340932 -0.000340932'
testdata['E_lambda300km']='-0.000332773 -0.000332773'
testdata['TEC']='40.6796      40.6796'  # 'TEC_above_140km'
testdata['Vi_IP']='0.00000      0.00000'
testdata['NmF2']='1.57999e+12  1.57999e+12'
testdata['HmF2']='300.000      300.000'

In [ ]:
checked=dict()
pos_ip=[ctipe._time[0],5.5,10,50]
# the Python algortihm calculates IP and uses that for regular interpolation in grid
# resulting in a slightly smaller height value thatn input.
# the required input height value resulting in 125 km in the output is 126.2 km. 
# THis shift in altitude / height may vary over the longitude-latitude grid.
# the IDL implemetnation uses the height value and interpolates at the 2 nearest (lon,lat) 
# grid stacks using the height values before applying the weights in longitude and latitude 
# to arrive at the final interpolatied value for each quantity
# numerical value for efield parameters are different between IDL and Python
# The efield values on the 37x73 grid are re-interpolated to the 91x20 plasma grid, a step not performed in Python
#
#pos_h=[ctipe._time[0],126200.,10,50]
pos_h=[ctipe._time[0],125000.,10,50]
print('pos:    %12.4g %12.4g %12.4g %12.4g' % (pos_ip[0],pos_ip[1],pos_ip[2],pos_ip[3]))
print('pos_h:  %12.4g %12.4g %12.4g %12.4g' % (pos_h[0],pos_h[1],pos_h[2],pos_h[3]))
ilev=ctipe.lev_density(ctipe,t=pos_h[0],z=pos_h[1],lon=pos_h[2],lat=pos_h[3])
#ilev=ctipe.lev_density_log(ctipe,t=pos_h[0],z=pos_h[1],lon=pos_h[2],lat=pos_h[3])
pos_h_ip=pos_h.copy() # lists may be assigned by reference unless using copy()
pos_h_ip[1]=ilev
print(pos_h_ip)
for var in ctipe:
    if var.name not in checked:  # prevent duplication
        checked[var.name]=1   #
        
        try:
            val_ip=np.squeeze(np.array(ctipe[var](pos_ip[0],pos_ip[1],pos_ip[2],pos_ip[3])))
        except:
            val_ip=np.squeeze(np.array(ctipe[var](pos_ip[0],pos_ip[2],pos_ip[3])))
        try:
            val_h=np.squeeze(np.array(ctipe[var](pos_h[0],ilev,pos_h[2],pos_h[3])))
        except:
            val_h=np.squeeze(np.array(ctipe[var](pos_h[0],pos_h[2],pos_h[3])))
#        try:
        val_h2=np.squeeze(np.array(ctipe.vert_interp(var,pos_h[0],pos_h[1],pos_h[2],pos_h[3])))
#        except:
#            val_h2=np.NaN # np.squeeze(np.array(ctipe[var](pos_h[0],pos_h[2],pos_h[3])))
        if var.name in testdata:
            print("%s %12.4g %12.4g %12.4g IDL: %s" % (var,val_ip,val_h,val_h2,testdata[var.name]))
        else:
            print("%s %12.4g %12.4g  IDL: %s" % (var,val_ip,val_h,"---"))
#          print(var,np.squeeze(np.array(ctipe[var](positions[0],positions[2],positions[3]))))
#            except:
#                val_ip=np.squeeze(np.array(ctipe[var](pos_ip[0],pos_ip[1],pos_ip[2],pos_ip[3])))
#                val_h=np.squeeze(np.array(ctipe[var](positions[0],ilev,positions[2],positions[3])))
#                print(var,val_ip,val_h)
##                print(var,np.squeeze(np.array(ctipe[var]([positions[0],positions[1],positions[2],positions[3]]))))

In [ ]:
ctipe.lev_density(ctipe,ctipe._time[0],125000.,30.,20.)

In [ ]:
ctipe._ctipe_density.variables['height'].data.shape

In [ ]:
print(pos_h)
ctipe.vert_interp('rho',pos_h[0],pos_h[1],pos_h[2],pos_h[3])

In [ ]:
lon=pos_h[2]
lat=pos_h[3]
lon_rep=np.repeat(lon,len(ctipe._ilev_density))
lat_rep=np.repeat(lat,len(ctipe._ilev_density))
xvec=np.vstack((ctipe._ilev_density,lat_rep,lon_rep)).T
print(lon_rep)
print(lat_rep)
print(ctipe._ilev_density)
print(xvec)

In [ ]:
z_levels=np.squeeze(np.log(ctipe.H(t = ctipe._time[0], lat = 30., lon = 20.)))

In [ ]:
ctipe._ilev_neutral

In [ ]:
from scipy.interpolate import RegularGridInterpolator, interp1d
level = interp1d(z_levels, ctipe._ilev_density, bounds_error=False)

In [ ]:
z_levels_abs=np.exp(z_levels)
level_abs = interp1d(z_levels_abs, ctipe._ilev_density, bounds_error=False)

In [ ]:
np.array(np.where(ctipe._lon_density <= 0.)).max()

In [ ]:
from scipy.interpolate import interp1d
import numpy as np
#@np.vectorize
def ctipe_vert_interp(self,varname,time,z,lat,lon):
    data=self[varname].data
    lons=self._lon_density
    lats=self._lat_density
    times=self._time
    #H=np.log(self['H'].data)
    H=self['H'].data

    ilon=np.array(np.where(lons[0:-1] <= lon)).max()
    if ilon == len(lons):
        ilon=ilon-1
    w0_lon=(lons[ilon+1]-lon)/(lons[ilon+1]-lons[ilon])
    print('ILON=%d W0=%f' %( ilon, w0_lon))

    ilat=np.array(np.where(lats[0:-1] <= lat)).max()
    if ilat == len(lats):
        ilat=ilat-1
    w0_lat=(lats[ilat+1]-lat)/(lats[ilat+1]-lats[ilat])
    print('ILAT=%d W0=%f' %( ilat, w0_lat) )

    itime=np.array(np.where(times <= time)).max()
# need to load next day's data and use itime=0 from there
    if itime == len(times):  
        itime=ilime-1                               
    w0_time=(times[itime+1]-time)/(times[itime+1]-times[itime])
    print('ITIME=%d W0=%f' %( itime, w0_time) )
    h_000=np.squeeze(H[itime,:,ilat,ilon])               
    d_000=np.squeeze(data[itime,:,ilat,ilon])
    int000=interp1d(h_000,d_000,bounds_error=False)
    print('Lons: ',lons.shape)
    print('Lats: ',lats.shape)
    print('Height: ',H.shape)
    print('Data:' ,data.shape)
    
    ilon1=ilon+1
    ilat1=ilat+1
    itime1=itime+1

    w1_lon=1.-w0_lon
    w1_lat=1-w0_lat
    w1_time=1-w0_time
    
    h_000=np.squeeze(H[itime,:,ilat,ilon])               
    d_000=np.squeeze(data[itime,:,ilat,ilon])
    int000=interp1d(h_000,d_000,bounds_error=False)
    
    h_001=np.squeeze(H[itime,:,ilat,ilon1])               
    d_001=np.squeeze(data[itime,:,ilat,ilon+1])
    int001=interp1d(h_000,d_000,bounds_error=False)
    
    h_010=np.squeeze(H[itime,:,ilat1,ilon])               
    d_010=np.squeeze(data[itime,:,ilat+1,ilon])
    int010=interp1d(h_000,d_000,bounds_error=False)
    
    h_011=np.squeeze(H[itime,:,ilat1,ilon1])               
    d_011=np.squeeze(data[itime,:,ilat+1,ilon+1])
    int011=interp1d(h_000,d_000,bounds_error=False)
    
    h_100=np.squeeze(H[itime+1,:,ilat,ilon])               
    d_100=np.squeeze(data[itime+1,:,ilat,ilon])
    int100=interp1d(h_000,d_000,bounds_error=False)
    
    h_101=np.squeeze(H[itime1,:,ilat,ilon1])               
    d_101=np.squeeze(data[itime1,:,ilat,ilon1])
    int101=interp1d(h_000,d_000,bounds_error=False)
    
    h_110=np.squeeze(H[itime1,:,ilat1,ilon])               
    d_110=np.squeeze(data[itime1,:,ilat1,ilon])
    int110=interp1d(h_000,d_000,bounds_error=False)
    
    h_111=np.squeeze(H[itime1,:,ilat1,ilon1])               
    d_111=np.squeeze(data[itime1,:,ilat1,ilon1])
    int111=interp1d(h_000,d_000,bounds_error=False)
    #zlog=np.log(z)                           
    zlog=z
    return(int000(zlog)*w0_lon*w0_lat*w0_time
        +int001(zlog)*w1_lon*w0_lat*w0_time
        +int010(zlog)*w0_lon*w1_lat*w0_time
        +int011(zlog)*w1_lon*w1_lat*w0_time
        +int100(zlog)*w0_lon*w0_lat*w1_time
        +int101(zlog)*w1_lon*w0_lat*w1_time
        +int110(zlog)*w0_lon*w1_lat*w1_time
        +int111(zlog)*w1_lon*w1_lat*w1_time
          )
    
    

In [ ]:
pos_h=[ctipe._time[0],125000.,10,50]
print(ctipe.vert_interp('H',ctipe._time[0],125000.,10,50.))

In [ ]:
print(ctipe_vert_interp(ctipe,'rho',ctipe._time[0],125000.,10,50.))


In [ ]:
H=np.array([[1.,2.,3.],[4.,5.,6.]])

In [ ]:
print(H.shape)